In [2]:
!pip install opencv-python

In [3]:
import numpy as np
import cv2

prototxt_path = '/content/colorization_deploy_v2.prototxt'
model_path = '/content/colorization_release_v2.caffemodel'
kernel_path = '/content/pts_in_hull.npy'
image_path = '/content/nasser.png'

# Load the pre-trained model
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# Load the points file
points = np.load(kernel_path)

# Resize the points
points = points.transpose().reshape(2, 313, 1, 1)

# Set the blobs for specific layers
net.getLayer(net.getLayerId("class8_ab")).blobs = [points.astype(np.float32)]
net.getLayer(net.getLayerId("conv8_313_rh")).blobs = [np.full([1, 313], 2.606, dtype="float32")]

# Load the black and white image and convert it to LAB color space
bw_image = cv2.imread(image_path)
normalized = bw_image.astype("float32") / 255.0
lab = cv2.cvtColor(normalized, cv2.COLOR_BGR2LAB)

# Resize the image to 224x224
resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]
L -= 50

# Set the input for the neural network
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

# Resize the output back to the original size
ab = cv2.resize(ab, (bw_image.shape[1], bw_image.shape[0]))
L = cv2.split(lab)[0]

# Combine the lightness with color to make the full image
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

# Convert the image back from LAB to BGR
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)

# Scale it back to 0-255 range
colorized = (255.0 * colorized).astype("uint8")

# Display the colorized image
plt.imshow(cv2.cvtColor(colorized, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
this is my code

NameError: name 'files' is not defined